In [ ]:
# offset outdated, use aimbot.ipynb or the offset_finder

In [1]:
import pymem

In [2]:
process = "deadlock.exe"

In [3]:
# Settings for the aiming
headshot_p = 0.5 # Probability of aiming at the head
aim_assist_p = 0.8 # Probability changing the user's aiming

In [4]:
offsets = {
    'local_player_controller': 0x2749860,
    'view_matrix': 0x2764410,
    'entity_list': 0x27BCF70,
    'camera_manager': 0x23C87D0,
    'schema_system_interface': 0x66880
}

In [5]:
pm = pymem.Pymem(process)

In [6]:
client_module = pymem.process.module_from_name(pm.process_handle, "client.dll")
client_base = client_module.lpBaseOfDll

In [7]:
entity_list = pm.read_longlong(client_base + offsets['entity_list'])

In [8]:
controller_base = pm.read_longlong(client_base + offsets['local_player_controller'])

In [9]:
pawn_handle = pm.read_longlong(controller_base + 0x60C)
list_entry = pm.read_longlong(entity_list + 0x8 * ((pawn_handle & 0x7FFF) >> 0x9) + 0x10)

In [10]:
camera = pm.read_longlong(client_base + offsets['camera_manager'] + 0x28)
# hex(camera + 0x44)
camera_x = pm.read_float(camera + 0x44)
camera_y = pm.read_float(camera + 0x48)
camera_z = pm.read_float(camera + 0x4C)
# print(f"camera_x: {camera_x}, camera_y: {camera_y}, camera_z: {camera_z}")

In [11]:
from enum import Enum

class HeroIds(Enum):
    Infernus = 1
    Seven = 2
    Vindicta = 3
    LadyGeist = 4
    Abrams = 6
    Wraith = 7
    McGinnis = 8
    Paradox = 10
    Dynamo = 11
    Kelvin = 12
    Haze = 13
    Holliday = 14
    Bebop = 15
    Calico = 16
    GreyTalon = 17
    MoAndKrill = 18
    Shiv = 19
    Ivy = 20
    Warden = 25
    Yamato = 27
    Lash = 31
    Viscous = 35
    Wrecker = 48
    Pocket = 50
    Mirage = 52
    Fathom = 53
    Dummy = 55
    Viper = 58
    Magician = 60
    Trapper = 61
    Raven = 62


In [12]:
def get_head_bone_index(hero_id):
    if hero_id == HeroIds.Abrams:
        return 7
    elif hero_id == HeroIds.Bebop:
        return 6
    elif hero_id == HeroIds.Dynamo:
        return 13
    elif hero_id == HeroIds.GreyTalon:
        return 17
    elif hero_id == HeroIds.Haze:
        return 13
    elif hero_id == HeroIds.Infernus:
        return 30
    elif hero_id == HeroIds.Ivy:
        return 13
    elif hero_id == HeroIds.Kelvin:
        return 12
    elif hero_id == HeroIds.LadyGeist:
        return 11
    elif hero_id == HeroIds.Lash:
        return 12
    elif hero_id == HeroIds.McGinnis:
        return 7
    elif hero_id == HeroIds.MoAndKrill:
        return 10  # 10
    elif hero_id == HeroIds.Paradox:
        return 8
    elif hero_id == HeroIds.Pocket:
        return 13
    elif hero_id == HeroIds.Seven:
        return 14
    elif hero_id == HeroIds.Shiv:
        return 13
    elif hero_id == HeroIds.Vindicta:
        return 7
    elif hero_id == HeroIds.Viscous:
        return 7
    elif hero_id == HeroIds.Warden:
        return 11
    elif hero_id == HeroIds.Wraith:
        return 7
    elif hero_id == HeroIds.Yamato:
        return 35
    elif hero_id == HeroIds.Mirage:
        return 8
    elif hero_id == HeroIds.Calico:
        return 13
    else:
        return None  # Return None or a default value if hero_id is not matched


In [13]:
def get_body_bone_index(hero_id):
    if hero_id == HeroIds.Abrams:
        return 4
    elif hero_id == HeroIds.Bebop:
        return 2
    elif hero_id == HeroIds.Dynamo:
        return 9
    elif hero_id == HeroIds.GreyTalon:
        return 10
    elif hero_id == HeroIds.Haze:
        return 7
    elif hero_id == HeroIds.Infernus:
        return 11
    elif hero_id == HeroIds.Ivy:
        return 9
    elif hero_id == HeroIds.Kelvin:
        return 9
    elif hero_id == HeroIds.LadyGeist:
        return 8
    elif hero_id == HeroIds.Lash:
        return 9
    elif hero_id == HeroIds.McGinnis:
        return 4
    elif hero_id == HeroIds.MoAndKrill:
        return 7  # 10
    elif hero_id == HeroIds.Paradox:
        return 5
    elif hero_id == HeroIds.Pocket:
        return 10
    elif hero_id == HeroIds.Seven:
        return 10
    elif hero_id == HeroIds.Shiv:
        return 9
    elif hero_id == HeroIds.Vindicta:
        return 4
    elif hero_id == HeroIds.Viscous:
        return 4
    elif hero_id == HeroIds.Warden:
        return 8
    elif hero_id == HeroIds.Wraith:
        return 5
    elif hero_id == HeroIds.Yamato:
        return 18
    elif hero_id == HeroIds.Mirage:
        return 6
    elif hero_id == HeroIds.Calico:
        return 10
    else:
        return None  # Return None or a default value if hero_id is not matched

In [14]:
def get_my_hero_id():
    hero_id = pm.read_short(controller_base + 0x8b8 + 0x1C) # PlayerDataGlobal + heroId
    hero_id = HeroIds(hero_id)
    return hero_id

In [15]:
my_hero_const = get_my_hero_id()
my_hero_const

<HeroIds.Viscous: 35>

In [16]:
# Function to return True if we should aim for the head in this instance
import random
def should_aim_for_head():
    return random.random() < headshot_p

def should_aim_assist():
    return random.random() < aim_assist_p

In [17]:
currently_aiming_for_head = False
aimbot_currently_on = True

# Start a thread that every 0.5 seconds will update currently_aiming_for_head based on should_aim_for_head()
import threading
import time

def update_aiming_for_head():
    global currently_aiming_for_head
    while True:
        currently_aiming_for_head = should_aim_for_head()
        if my_hero_const == HeroIds.Seven:
            time.sleep(1)
        else:
            time.sleep(0.5)

thread = threading.Thread(target=update_aiming_for_head)
thread.start()

def update_aim_assist():
    global aimbot_currently_on
    while True:
        aimbot_currently_on = should_aim_assist()
        time.sleep(0.1)

thread2 = threading.Thread(target=update_aim_assist)
# thread2.start()

In [18]:
def get_index(i):
    address_base = pm.read_longlong(entity_list + 0x8 * ((i & 0x7FFF) >> 0x9) + 0x10)
    controller_base = pm.read_longlong(address_base + 120 * (i & 0x1FF))
    if i == 0:
        controller_base = pm.read_longlong(client_base + offsets['local_player_controller'])

    hero_id = pm.read_short(controller_base + 0x8b8 + 0x1C)
    hero_id = HeroIds(hero_id)
        
    pawn_handle = pm.read_longlong(controller_base + 0x878) # CPlayerController + m_pawnHandle
    list_entry = pm.read_longlong(entity_list + 0x8 * ((pawn_handle & 0x7FFF) >> 0x9) + 0x10)
    pawn = pm.read_longlong(list_entry + 0x78 * (pawn_handle & 0x1FF))

    camera_services = pm.read_longlong(pawn + 0xf80) # C_BasePlayerPawn + CPlayer_CameraServices
    aim_angle = pm.read_float(camera_services + 0x40) #m_vecPunchAngle

    health = pm.read_int(pawn + 0x354)

    game_scene_node = pm.read_longlong(pawn + 0x330)

    if currently_aiming_for_head:
        bone_index = get_head_bone_index(hero_id)
    else:
        bone_index = get_body_bone_index(hero_id)

    team_num = pm.read_int(controller_base + 0x3f3)
    dormant = False#pm.read_bool(game_scene_node + 0xEF)
    
    pos_addr = game_scene_node + 0xD0
    # Read 4 floats from the pos_addr
    pos = pm.read_float(pos_addr)
    pos2 = pm.read_float(pos_addr + 4)
    pos3 = pm.read_float(pos_addr + 8)

    pos_vector = (pos, pos2, pos3 + 70) # Add a defau;t +75 to aim for head for default models

    try:
        if bone_index is not None:
            bone_array = pm.read_longlong(game_scene_node + 0x170 + 0x80)
            head_vector_start = bone_array + bone_index * 32
            head_vector = (pm.read_float(head_vector_start), pm.read_float(head_vector_start + 4), pm.read_float(head_vector_start + 8))
            if i > 1 and bone_index is not None:
                pos_vector = head_vector
    except:
        pass

    return pawn, pos_vector, team_num, dormant, health, aim_angle, game_scene_node, hero_id

In [19]:
get_index(0)

(1871348432896,
 (2349.90625, -979.4375, 62.0),
 2,
 False,
 700,
 0.0,
 1861906530432,
 <HeroIds.Viscous: 35>)

In [20]:
import math

def calculate_camera_rotation(v1, v2):
    """
    Calculate the yaw and pitch required to point from vector v1 to vector v2.

    Parameters:
    v1: tuple or list of (x1, y1, z1)
    v2: tuple or list of (x2, y2, z2)

    Returns:
    yaw: The yaw angle in radians
    pitch: The pitch angle in radians
    """
    # Compute the vector difference
    dx = v2[0] - v1[0]
    dy = v2[1] - v1[1]
    dz = v2[2] - v1[2]
    
    # Compute yaw and pitch
    yaw = math.atan2(dy, dx)
    pitch = math.atan2(dz, math.hypot(dx, dy))
    
    return math.degrees(yaw), math.degrees(pitch)

In [21]:
def calculate_camera_rotation_with_roll(v1, v2, target_roll):
    """
    Calculate the yaw, pitch, and roll required to point from vector v1 to vector v2,
    considering the effect of a specified roll.

    Parameters:
    v1: tuple or list of (x1, y1, z1) - the starting position
    v2: tuple or list of (x2, y2, z2) - the target position
    target_roll: The desired roll angle in degrees

    Returns:
    yaw: The yaw angle in degrees
    pitch: The pitch angle in degrees
    roll: The roll angle in degrees (equal to target_roll)
    """
    # Compute the vector difference
    dx = v2[0] - v1[0]
    dy = v2[1] - v1[1]
    dz = v2[2] - v1[2]
    
    # Convert target roll to radians
    roll_rad = math.radians(target_roll)
    
    # Apply roll to the direction vector (dx, dy, dz)
    # Rotate around the forward axis (Z-axis) by roll
    dx_rot = dx * math.cos(roll_rad) - dy * math.sin(roll_rad)
    dy_rot = dx * math.sin(roll_rad) + dy * math.cos(roll_rad)
    
    # Compute yaw and pitch based on the rotated coordinates
    yaw = math.atan2(dy_rot, dx_rot)
    pitch = math.atan2(dz, math.hypot(dx_rot, dy_rot))

    return math.degrees(yaw), math.degrees(pitch), float(target_roll)

In [22]:
def get_camera_pos():
    # Get vector from camera + 0x38
    camera_pos = (pm.read_float(camera + 0x38), pm.read_float(camera + 0x3C), pm.read_float(camera + 0x40))
    return camera_pos

In [23]:
def calculate_new_camera_angles(current_yaw, current_pitch, desired_yaw, desired_pitch, max_change):
    # Calculate the new camera yaw and pitch
    # We do not want to instantly update to the desired values, but instead gradually move towards them
    # So we limit the movement to max_change degrees per frame
    # And this function returns the next camera position based on the current position and desired position
    
    # It has to handle negative values. For example if yaw is 350 and desired yaw is 10, it should move 20 degrees to the right instead of 340 degrees to the left
    # So we calculate the difference in both directions and choose the smaller one

    # Calculate the difference in both directions
    diff1 = desired_yaw - current_yaw
    diff2 = desired_yaw - current_yaw + 360
    diff3 = desired_yaw - current_yaw - 360

    # Choose the smallest difference
    yaw_diff = min(diff1, diff2, diff3, key=abs)

    # Calculate the difference in both directions for pitch
    diff1 = desired_pitch - current_pitch
    diff2 = desired_pitch - current_pitch + 360
    diff3 = desired_pitch - current_pitch - 360

    # Choose the smallest difference
    pitch_diff = min(diff1, diff2, diff3, key=abs)

    # Limit the movement to max_change degrees per frame
    yaw_diff = max(-max_change, min(max_change, yaw_diff))
    pitch_diff = max(-max_change, min(max_change, pitch_diff))

    # Calculate the new yaw and pitch
    new_yaw = current_yaw + yaw_diff
    new_pitch = current_pitch + pitch_diff

    return new_yaw, new_pitch

In [24]:
import keyboard
import time

def warden_macro():
    # Function that will be ran in the background if we are playing warden
    # It checks if we pressed F
    # And if we did, it will press C automatically after 0.1 seconds

    while True:
        # Check if the F key was pressed on the keyboard
        if keyboard.is_pressed('f'):
            # If it was, wait for 0.1 seconds
            time.sleep(0.1)
            # And press the C key
            keyboard.press_and_release('c')
        time.sleep(0.01)
        


In [25]:
def mo_and_krill_macro():
    # For this while the user is pressing r, we auto repeat it every 0.05 seconds
    while True:
        if keyboard.is_pressed('r'):
            time.sleep(0.05)
            keyboard.press_and_release('r')
        time.sleep(0.01)

In [26]:
import time
import win32api
import threading
import ctypes
import time
import keyboard

aim_enabled = True
force_aim = False # Lock aim even if left mouse button is not pressed

my_hero_id = get_my_hero_id()

print(f"Hero ID: {my_hero_id}")

# if my_hero_id == HeroIds.Warden:
#     threading.Thread(target=warden_macro).start()

if my_hero_id == HeroIds.MoAndKrill:
    threading.Thread(target=mo_and_krill_macro).start()

hold_down_left_click = False

target_select_type = "fov" # can be 'distance' or 'fov'

if my_hero_id == HeroIds.Vindicta or my_hero_id == HeroIds.GreyTalon:
    target_select_type = "fov"

# if my_hero_id == HeroIds.Abrams:
#     target_select_type = "distance"

# Variable to hold the index of the enemy that we are currently locked on
locked_on_enemy = None

while False:
    # if right click is pressed, press left click
    if not hold_down_left_click and win32api.GetKeyState(0x02) < 0:
        hold_down_left_click = True
        ctypes.windll.user32.mouse_event(2, 0, 0, 0, 0)

    if hold_down_left_click and win32api.GetKeyState(0x02) >= 0:
        hold_down_left_click = False
        ctypes.windll.user32.mouse_event(4, 0, 0, 0, 0)

    # Toggle aim_enabled if mouse 3 was pressed
    # if win32api.GetKeyState(0x04) < 0:
    #     aim_enabled = not aim_enabled
    #     print(f"Aimbot {'enabled' if aim_enabled else 'disabled'}")
    #     time.sleep(0.1)


    # If "r" was pressed
    if win32api.GetKeyState(0x52) < 0 and my_hero_id == HeroIds.Vindicta and not force_aim:
        # We do a special loop where we continue to aim for 1 second before firing a left mouse click
        # Start a background thread that will fire a left mouse click after 1 second

        # If left mouse is pressed, release it
        if win32api.GetKeyState(0x01) < 0:
            ctypes.windll.user32.mouse_event(4, 0, 0, 0, 0)

        force_aim = True

        def fire_left_click():
            global force_aim
            time.sleep(0.2)
            ctypes.windll.user32.mouse_event(4, 0, 0, 0, 0)
            time.sleep(0.01)
            ctypes.windll.user32.mouse_event(2, 0, 0, 0, 0)
            
            time.sleep(0.01)
            ctypes.windll.user32.mouse_event(4, 0, 0, 0, 0)
            time.sleep(0.05)
            # Press and release r
            keyboard.press_and_release('r')
            time.sleep(0.15)
            force_aim = False
            
        
        threading.Thread(target=fire_left_click).start()
        print("Firing in 1 second")

    # If playing grey talon and q was pressed
    if win32api.GetKeyState(0x51) < 0 and (my_hero_id == HeroIds.GreyTalon or my_hero_id == HeroIds.Yamato) and not force_aim:
        force_aim = True

        # Start a thread to turn off force aim after 0.5 seconds
        def turn_off_force_aim():
            global force_aim
            time.sleep(0.5) if my_hero_id == HeroIds.GreyTalon else time.sleep(1.4)
            # Release then press and release left mouse
            ctypes.windll.user32.mouse_event(4, 0, 0, 0, 0)
            time.sleep(0.05)
            ctypes.windll.user32.mouse_event(2, 0, 0, 0, 0)
            time.sleep(0.05)
            ctypes.windll.user32.mouse_event(4, 0, 0, 0, 0) 
            force_aim = False

        threading.Thread(target=turn_off_force_aim).start()

    # If "1" is pressed and playing grey talon, then press q
    if win32api.GetKeyState(0x31) < 0 and (my_hero_id == HeroIds.GreyTalon or my_hero_id == HeroIds.Yamato):
        keyboard.press_and_release('q')

    # If "r" is pressed and playing paradox, then press e
    if win32api.GetKeyState(0x52) < 0 and my_hero_id == HeroIds.Paradox:
        time.sleep(0.02)
        keyboard.press_and_release('e')

    closest_enemy = None
    closest_enemy_health = None
    closest_distance = None
    closest_camera_distance = None
    try:
        _, my_pos_vector, my_team_num, my_dormant, my_health, my_aim_angle, _ = get_index(0)
    except:
        time.sleep(0.1)
        continue

    camera_pos_vector = get_camera_pos()
    # First we get the current yaw and pitch from memory
    current_yaw = pm.read_float(camera + 0x48)
    current_pitch = pm.read_float(camera + 0x44)

    # If locked_on_enemy is not None and left mouse button is not pressed, we reset the locked_on_enemy to None
    if locked_on_enemy is not None and win32api.GetKeyState(0x01) >= 0:
        locked_on_enemy = None
    
    if locked_on_enemy is None:
        # We need to find a locked on enemy
        for i in range(1, 16):
            try:
                try:
                    _, enemy_pos_vector, enemy_team_num, enemy_dormant, enemy_health, _, _ = get_index(i)
                except:
                    continue
                if my_team_num == enemy_team_num or my_dormant or enemy_dormant or enemy_health <= 0:
                    continue
            except Exception as e:
                continue

            if target_select_type == "distance":
                distance = math.sqrt((my_pos_vector[0] - enemy_pos_vector[0])**2 + (my_pos_vector[1] - enemy_pos_vector[1])**2 + (my_pos_vector[2] - enemy_pos_vector[2])**2)
                if closest_enemy is None or distance < closest_distance:
                    closest_enemy = enemy_pos_vector
                    closest_distance = distance
                    closest_enemy_health = enemy_health
            elif target_select_type == "fov":
                yaw, pitch = calculate_camera_rotation(camera_pos_vector, enemy_pos_vector)
                pitch = pitch * -1 # Invert pitch as needed by game

                yaw_change = min(abs(yaw - current_yaw), abs(yaw - current_yaw + 360), abs(yaw - current_yaw - 360))
                pitch_change = min(abs(pitch - current_pitch), abs(pitch - current_pitch + 360), abs(pitch - current_pitch - 360))

                if closest_camera_distance is None or yaw_change + pitch_change < closest_camera_distance:
                    closest_camera_distance = yaw_change + pitch_change
                    closest_enemy = enemy_pos_vector
                    closest_enemy_health = enemy_health

                    # Update the closest_distance also
                    closest_distance = math.sqrt((my_pos_vector[0] - enemy_pos_vector[0])**2 + (my_pos_vector[1] - enemy_pos_vector[1])**2 + (my_pos_vector[2] - enemy_pos_vector[2])**2)

                    locked_on_enemy = i

        if closest_enemy is None:
            # time.sleep(0.1)
            continue
    else:
        try:
            closest_enemy = get_index(locked_on_enemy)[1]
        except:
            time.sleep(0.1)
            continue

    if closest_enemy is None:
        # time.sleep(0.1)
        continue

    camera_pos_vector = get_camera_pos()

    # Add randomness to the coordinates
    # random_range = 25
    # if not force_aim:
    #     closest_enemy = (closest_enemy[0] + random.uniform(-random_range, random_range), closest_enemy[1] + random.uniform(-random_range, random_range), closest_enemy[2] + random.uniform(-random_range, random_range))

    yaw, pitch = calculate_camera_rotation(camera_pos_vector, closest_enemy)

    
    # update_yaw, update_pitch = calculate_new_camera_angles(current_yaw, current_pitch, yaw, pitch * -1, 0.035)

    # For vindicta only update aim if angles are within a threshold
    aimcond = True if my_hero_id != HeroIds.Vindicta else (abs(yaw - current_yaw) < 20 or abs(yaw - current_yaw + 360) < 20 or abs(yaw - current_yaw - 360) < 20) and (abs(pitch * -1 - current_pitch) < 20 or abs(pitch * -1 - current_pitch + 360) < 20 or abs(pitch * -1 - current_pitch - 360) < 20)

    # Check if left mouse button is down
    if (win32api.GetKeyState(0x01) < 0 or force_aim) and aim_enabled and not hold_down_left_click and (aimbot_currently_on or force_aim):
        if aimcond:
            pm.write_float(camera + 0x44, pitch * -1 - my_aim_angle)
            pm.write_float(camera + 0x48, yaw)
            # pm.write_float(camera + 0x44, update_pitch)
            # pm.write_float(camera + 0x48, update_yaw)
            pm.write_float(camera + 0x4C, 0.)
    else:
        # time.sleep(0.1)
        continue

    # print(f"Current yaw: {current_yaw}, Current pitch: {current_pitch}")
    # print(f"Yaw: {yaw}, Pitch: {pitch}")
    # print(f"Update Yaw: {update_yaw}, Update Pitch: {update_pitch}")
    # print()

    # break


Hero ID: HeroIds.Viscous


In [ ]:
import pygame
import numpy as np
import ctypes

# Initialize pygame for the overlay
pygame.init()
infoObject = pygame.display.Info()
screen = pygame.display.set_mode((infoObject.current_w, infoObject.current_h), pygame.NOFRAME | pygame.SRCALPHA)
pygame.display.set_caption("ESP Overlay")
clock = pygame.time.Clock()

hwnd = pygame.display.get_wm_info()["window"]
ctypes.windll.user32.SetWindowLongW(hwnd, -20, ctypes.windll.user32.GetWindowLongW(hwnd, -20) | 0x80000 | 0x20)  # WS_EX_LAYERED | WS_EX_TRANSPARENT
ctypes.windll.user32.SetLayeredWindowAttributes(hwnd, 0, 255, 1)  # LWA_ALPHA for alpha transparency


# Function to convert 3D coordinates to 2D screen coordinates
def world_to_screen(view_matrix, pos, width, height):
    """
    Converts 3D world coordinates to 2D screen coordinates using the view matrix.
    """
    clip_coords = [
        pos[0] * view_matrix[0][0] + pos[1] * view_matrix[0][1] + pos[2] * view_matrix[0][2] + view_matrix[0][3],
        pos[0] * view_matrix[1][0] + pos[1] * view_matrix[1][1] + pos[2] * view_matrix[1][2] + view_matrix[1][3],
        pos[0] * view_matrix[2][0] + pos[1] * view_matrix[2][1] + pos[2] * view_matrix[2][2] + view_matrix[2][3],
        pos[0] * view_matrix[3][0] + pos[1] * view_matrix[3][1] + pos[2] * view_matrix[3][2] + view_matrix[3][3],
    ]

    if clip_coords[3] < 0.1:  # Avoid division by zero
        return None

    # Perform perspective division
    ndc_x = clip_coords[0] / clip_coords[3]
    ndc_y = clip_coords[1] / clip_coords[3]

    # Convert to screen coordinates
    screen_x = int((ndc_x + 1) * (width / 2))
    screen_y = int((1 - ndc_y) * (height / 2))  # Invert Y-axis for screen space

    return screen_x, screen_y


# Function to draw skeleton
def draw_skeleton(screen, entity_bones, color=(255, 0, 0)):
    i = 0
    for bone_start, bone_end in entity_bones:
        start_pos = world_to_screen(view_matrix, bone_start, infoObject.current_w, infoObject.current_h)
        end_pos = world_to_screen(view_matrix, bone_end, infoObject.current_w, infoObject.current_h)
        if start_pos and end_pos:
            pygame.draw.line(screen, color, start_pos, end_pos, 2)
            # Draw the text name over it
            font = pygame.font.Font(None, 18)
            text = font.render(str(i), True, (255, 255, 255))
            screen.blit(text, (start_pos[0], start_pos[1] - 10))
        i += 1       

# Main loop for ESP
running = True
while running:
    screen.fill((0, 0, 0, 0))  # Clear screen with transparency

    # Read view matrix
    view_matrix = pm.read_bytes(client_base + offsets['view_matrix'], 16 * 4)
    view_matrix = np.frombuffer(view_matrix, dtype=np.float32).reshape(4, 4)


    for i in range(1, 16):
        try:
            _, entity_pos, _, _, _, _, game_scene_node, hero_id = get_index(i)
            bone_array = pm.read_longlong(game_scene_node + 0x170 + 0x80)

            # Collect all bone positions for the entity
            entity_bones = []
            for bone_index in range(0, 64):  # Assuming 16 bones per entity
                bone_start = (
                    pm.read_float(bone_array + bone_index * 32),
                    pm.read_float(bone_array + bone_index * 32 + 4),
                    pm.read_float(bone_array + bone_index * 32 + 8),
                )
                # Connect bones (You need a predefined bone connection list for proper skeleton structure)
                entity_bones.append((bone_start, bone_start))  

            # Draw skeleton for the entity
            draw_skeleton(screen, entity_bones)

            # Todo: draw the hero id

        except Exception as e:
            continue

    # Update display
    pygame.display.flip()
    clock.tick(60)  # Limit to 60 FPS

    # Handle quit event
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            pygame.quit()
            break


pygame 2.6.1 (SDL 2.28.4, Python 3.12.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


: 